In [1]:
import pandas as pd
from tqdm import tqdm, tqdm_notebook
import warnings;
warnings.filterwarnings('ignore');
tqdm_notebook().pandas() #gives progress_apply bar

# Read in datasets
sb = pd.read_csv("sbro_scrape.csv")
sid = pd.read_csv("game_infos.csv")

## Ignore odds portal dataset - no over/under lines
#op = pd.read_csv("oddsportal_scrape.csv")

In [2]:
#Clean SB Data - only run once!

#no duplicates... but good practice
sb.drop_duplicates(inplace=True)

#insert and rename columns
sb.insert(5, "away_pitcher", None)
sb.insert(19, "open_under", None)
sb.insert(22, "close_under", None)
sb.rename(columns={'Date': 'date',
                   'Rot': 'start_time',
                   'Team': 'home_team',
                   'Pitcher': 'home_pitcher',
                   'Open OU Odds': 'open_odds',
                   'O Over': 'open_over',
                   'C Over': 'close_over',
                   'Close OU': 'close_odds'}, inplace=True)

# sloppy pitcher/odds rearrange... yes, you shouldn't modify in loops
r=0
for i, row in tqdm_notebook(sb.iterrows(), total=sb.shape[0]):  
    if r==0: ## away row
        try:
            ap = sb.at[i,'home_pitcher']
            hp = sb.at[i+1,'home_pitcher']
            ou = sb.at[i+1,'open_over']
            oo = sb.at[i,'open_over']
            cu = sb.at[i+1,'close_over']
            co = sb.at[i,'close_over']
        except:
            print("broke")
        sb.at[i,'home_pitcher'] = hp
        sb.at[i,'away_pitcher'] = ap
        sb.at[i,'open_under'] = ou
        sb.at[i,'close_under'] = cu
    if r==1: ## home row
        sb.at[i,'away_pitcher'] = ap
        sb.at[i,'open_over'] = oo
        sb.at[i,'open_under'] = ou
        sb.at[i,'close_over'] = co
        sb.at[i,'close_under'] = cu
    r = 1 if r == 0 else 0

# drop all visitor rows
sb = sb[sb.VH == 'H']

# drop innings columns
sb = sb.drop(['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','Final','Open','Close'], axis='columns')
    
#convert to datetime object
sb['date'] = sb['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

#renaming like teams
sb["home_team"].replace("LOS","LAD",inplace=True) 
sb["home_team"].replace("HOW","HOU",inplace=True) #fix one record

#sb.to_csv(r'sbro_scrape2.csv', index=False)

In [3]:
##Ignoring Odds Portal dataset - no over/under lines

##no duplicates... but good practice
# op.drop_duplicates(inplace=True)

##convert to datetime object
# op['Date'] = op['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

##renaming like teams
# op.replace("Florida Marlins","Miami Marlins",inplace=True)

##prep for merge
# op.rename({'Date': 'date'}, axis=1, inplace=True)

##op.to_csv(r'op_scrape2.csv', index=False)

In [4]:
#CLEAN SID Data - only run once!

#drop duplicate games
print("**TODO: check you are happy with the start_times being dropped**")
sid.drop_duplicates(subset=sid.columns.difference(['start_time']),inplace=True)

#insert/rename columns
sid.rename({'date_played': 'date'}, axis=1, inplace=True)

#convert to datetime object
sid['date'] = sid['date'].progress_apply(lambda x: pd.to_datetime(str(x), format='%d-%b-%y'))

#renaming like teams
sid.replace("Anaheim Angels","Los Angeles Angels",inplace=True)
sid.replace("Florida Marlins","Miami Marlins",inplace=True)
sid.replace("Los Angeles Angels of Anaheim","Los Angeles Angels",inplace=True)
sid.replace("Montreal Expos","Washington Nationals",inplace=True)
sid.replace("Tampa Bay Devil Rays","Tampa Bay Rays",inplace=True)

#sid.to_csv(r'game_infos2.csv', index=False)

**TODO: check you are happy with the start_times being dropped**


In [5]:
# Dicts to align teams across data sets

sid_key = {
    "Arizona Diamondbacks" : 0,
    "Atlanta Braves": 1,
    "Baltimore Orioles": 2,
    "Boston Red Sox": 3,
    "Chicago Cubs": 4,
    "Chicago White Sox": 5,
    "Cincinnati Reds": 6,
    "Cleveland Indians": 7,
    "Colorado Rockies": 8,
    "Detroit Tigers": 9,
    "Houston Astros": 10,
    "Kansas City Royals": 11,
    "Los Angeles Angels": 12,
    "Los Angeles Dodgers": 13,
    "Miami Marlins": 14,
    "Milwaukee Brewers": 15,
    "Minnesota Twins": 16,
    "New York Mets": 17,
    "New York Yankees": 18,
    "Oakland Athletics": 19,
    "Philadelphia Phillies": 20,
    "Pittsburgh Pirates": 21,
    "San Diego Padres": 22,
    "San Francisco Giants": 23,
    "Seattle Mariners": 24,
    "St. Louis Cardinals": 25,
    "Tampa Bay Rays": 26,
    "Texas Rangers": 27,
    "Toronto Blue Jays": 28,
    "Washington Nationals": 29
}
#     0: "Anaheim Angels", convert to "Los Angeles Angels"
#     10: "Florida Marlins", convert to "Miami Marlins"
#     15: "Los Angeles Angels of Anaheim", convert to "Los Angeles Angels"
#     17: "Montreal Expos", convert to "Washington Nationals"
#     30: "Tampa Bay Devil Rays", convert to "Tampa Bay Rays"

sb_sid = {
    "ARI":0,
    "ATL":1,
    "BAL":2,
    "BOS":3,
    "CIN":6,
    "CLE":7,
    "COL":8,
    "CUB":4,
    "CWS":5,
    "DET":9,
    "HOU":10,
    "KAN":11,
    "LAA":12,
    "LAD":13,
    "MIA":14,
    "MIL":15,
    "MIN":16,
    "NYM":17,
    "NYY":18,
    "OAK":19,
    "PHI":20,
    "PIT":21,
    "SDG":22,
    "SEA":24,
    "SFO":23,
    "STL":25,
    "TAM":26,
    "TEX":27,
    "TOR":28,
    "WAS":29
}
#     "LOS":13, #Old Dodgers

# op_sid = {
#     "American League": 999,
#     "Arizona Diamondbacks": 0,
#     "Atlanta Braves": 1,
#     "Australia": 999,
#     "Baltimore Orioles": 2,
#     "Boston Red Sox": 3,
#     "Campeche": 999, 
#     "Chicago Cubs": 4,
#     "Chicago White Sox": 5,
#     "Cincinnati Reds": 6,
#     "Cleveland Indians": 7,
#     "Colorado Rockies": 8,
#     "Detroit Tigers": 9,
#     "El Paso Chihuahuas": 999,
#     "Houston Astros": 10,
#     "Kansas City Royals": 11,
#     "Los Angeles Angels": 12,
#     "Los Angeles Dodgers": 13,
#     "Miami Marlins": 14,
#     "Milwaukee Brewers": 15,
#     "Minnesota Twins": 16,
#     "National League": 999,
#     "New York Mets": 17,
#     "New York Yankees": 18, 
#     "Oakland Athletics": 19, 
#     "Philadelphia Phillies": 20,
#     "Pittsburgh Pirates": 21,
#     "Sacramento River Cats": 999,
#     "San Diego Padres": 22,
#     "San Francisco Giants": 23,
#     "Seattle Mariners": 24, 
#     "St.Louis Cardinals": 25, 
#     "Tabasco": 999, 
#     "Tampa Bay Rays": 26,
#     "Texas Rangers": 27,
#     "Toronto Blue Jays": 28,
#     "Washington Nationals": 29
# }
# #     10: "Florida Marlins", convert to "Miami Marlins"

In [6]:
import datetime

# sloppy UID creator... yes you shouldn't modify in loops - shoot me
def dh(df, i):
    idx = set(df.index[df['date']==df['date'][i]]) & set(df.index[df['home_team']==df['home_team'][i]])
    i2  = idx - set([i])
    return 1 if df.loc[i].start_time > df.loc[i2.pop()].start_time else 0

def make_UID(df, lookup_dict):
    #UID: yymmdd_team_dhpos
    if 'UID' not in df.columns:
        df.insert(0, "UID", None)   
    dh_table = df.groupby(['date','home_team']).size()
    for i, row in tqdm_notebook(df.iterrows(), total=df.shape[0]): 
        team = lookup_dict[row["home_team"]]
        date = row['date'].strftime("%y%m%d")
        if dh_table[row['date']][row['home_team']] < 2:
            dhpos = 0
        else:
            dhpos = dh(df, i)
            #print(f'{i}, {team}, {date}, {dhpos}')
        df.at[i,'UID'] = f'{date}_{team}_{dhpos}'

In [7]:
## sanity check double headers
# temp = sid.groupby(['date','home_team']).size().reset_index().rename(columns={0:'count'})
# temp.groupby('count').count()
# for i, r in temp.iterrows():
#     if temp.at[i,'count'] == 2:
#         print (f'2: {r["home_team"]}, {r["date"]}')
#     None

In [8]:
make_UID(sid, sid_key)

In [9]:
make_UID(sb, sb_sid)

In [10]:
# Issue to dig into - there shouldn't be non-unique IDs
print("**TODO: Check why there are non-unique IDs**\n")
temp = sid.groupby(['UID','date']).size().reset_index().rename(columns={0:'count'})
print(temp.groupby('count').count(),"\n")
for i, r in temp.iterrows():
    if temp.at[i,'count'] > 1:
        print (f'i:{i} {r["UID"]}, {r["date"]}')
    None

**TODO: Check why there are non-unique IDs**

         UID   date
count              
1      48568  48568
2         11     11 

i:5217 020428_4_0, 2002-04-28 00:00:00
i:5483 020518_7_0, 2002-05-18 00:00:00
i:5966 020624_27_0, 2002-06-24 00:00:00
i:6216 020715_11_0, 2002-07-15 00:00:00
i:6818 020827_6_0, 2002-08-27 00:00:00
i:6872 020831_4_0, 2002-08-31 00:00:00
i:7006 020910_18_0, 2002-09-10 00:00:00
i:29143 120328_19_0, 2012-03-28 00:00:00
i:29144 120329_19_0, 2012-03-29 00:00:00
i:44607 180605_16_0, 2018-06-05 00:00:00
i:48350 190912_9_0, 2019-09-12 00:00:00


In [11]:
# merge
sb_col = ['UID','home_pitcher', 'away_pitcher', 'open_odds', 'open_under', 
          'open_over', 'close_odds', 'close_under', 'close_over']
m = pd.merge(sid, sb[sb_col],
             on='UID', 
             how='left')#,
             #validate="one_to_many")

m.to_csv(r'output.csv', index=False)

In [12]:
b  =len(set(sid.UID) & set(sb.UID)) # UIDs in both
d1 =len(set(sb.UID) - set(sid.UID)) # sb not in sid
d2 =len(set(sid.UID) - set(sb.UID))
#(set(sb.UID) & set(sid.UID))
print(f'Summary\n{m.shape} merged dataset\n{b} rows in common\n{d2} rows not in SB\n{d1} SB rows not used')

Summary
(48590, 153) merged dataset
24270 rows in common
24309 rows not in SB
377 SB rows not used
